In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'PredictionCode/'
import os
import sys
sys.path.append(base_dir)
print(os.getcwd())
print(os.listdir())
os.chdir(base_dir)
print(os.getcwd())
print(os.listdir())

In [ ]:

from torch import nn
import torch
import copy
from torch.utils import data
import numpy as np
import random
import torchvision
from torchvision import models, transforms
from util.dataset import TrainDataset, TestDataset, ValDataset
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('\ndevice = {0}\n'.format(device))


#instantiate model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
        
loss_function = nn.CrossEntropyLoss()



def instance_segmentation_api(im):
    threshold = 0.75
    img = np.array(im)
    img = transforms.ToTensor()(img).to(device)
    out = model([img])
    # scores + masks
    scores = out[0]['scores'].detach().cpu()
    mask = out[0]['masks'].detach().cpu()
    
    if(len(mask) == 0):
        return im
    color_array = np.zeros([mask[0].shape[1], mask[0].shape[2],3], dtype=np.uint8)
    best_idx = np.where(scores>threshold) 
    if len(best_idx)>0:
       bgr_img = im
       ax = plt.gca()
       for id, b in enumerate(best_bboxes):
           if classes[id].item() == 1:
              found = mask[id][0].detach().clone().cpu().numpy()
              color_array[found<0.5] = [255,255,255]

       added_image = cv2.addWeighted(bgr_img, 0.5, color_array, 1, 0)       
       return added_image



In [ ]:
import os
import torch
from torch.utils import data
from torchvision import transforms
from PIL import Image
import numpy as np
import re
import random
import shutil
import numpy as np
import cv2
import os
from os import listdir
from os.path import isfile, join


def transform_clip(video):
    t_ = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize((256, 256)),
                                transforms.ToTensor(),
                                ])
    new_clip = []
    for i in range(len(video)):
        frame = t_(np.array(video[i]))
        new_clip.append(frame)
    return torch.stack(new_clip)

def video_to_array():
        directories = [x[0] for x in os.walk('./hmdb51_org')]
        train_count = 0
        test_count = 0
        val_count = 0
        count = 0
        if os.path.exists('./PredData/train_mask/'):
            shutil.rmtree('./PredData/train_mask/')
        os.mkdir('./PredData/train_mask/')

        if os.path.exists('./PredData/train_labels_mask/'):
            shutil.rmtree('./PredData/train_labels_mask/')
        os.mkdir('./PredData/train_labels_mask/')

        if os.path.exists('./PredData/val_mask/'):
            shutil.rmtree('./PredData/val_mask/')
        os.mkdir('./PredData/val_mask/')

        if os.path.exists('./PredData/val_labels_mask/'):
            shutil.rmtree('./PredData/val_labels_mask/')
        os.mkdir('./PredData/val_labels_mask/')

        if os.path.exists('./PredData/test_mask/'):
            shutil.rmtree('./PredData/test_mask/')
        os.mkdir('./PredData/test_mask/')

        if os.path.exists('./PredData/test_labels_mask/'):
            shutil.rmtree('./PredData/test_labels_mask/')
        os.mkdir('./PredData/test_labels_mask/')

        
        label = 0
        labels = ['handstand', 'stand', 'smoke', 'somersault',
                     'clap', 'smile', 'sit', 'wave', 'laugh', 'cartwheel']
        for dir in directories:
            skip = True
            for l in labels:
                if dir.__contains__(l):
                    skip = False
            if skip:
                continue
            print(dir)
            videos = [f for f in listdir(dir) if isfile(join(dir, f))]
            for video in videos:
                arr = []
                cap = cv2.VideoCapture(join(dir, video))
                while cap.isOpened():
                    ret, frame = cap.read()
                    if frame is None:
                        break
                    print(np.shape(frame))
                    frame = instance_segmentation_api(frame)
                    arr.append(frame)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    if len(arr) == 70:
                        if count % 10 == 0:
                            torch.save(transform_clip(arr), './PredData/test_mask/' + str(test_count) + '.pt')
                            torch.save(torch.tensor([label]), './PredData/test_labels_mask/' + str(test_count) + '.pt')
                            test_count +=1
                        elif count % 10 == 1:
                            torch.save(transform_clip(arr), './PredData/val_mask/' + str(val_count) + '.pt')
                            torch.save(torch.tensor([label]), './PredData/val_labels_mask/' + str(val_count) + '.pt')
                            val_count +=1
                        else:
                            torch.save(transform_clip(arr), './PredData/train_mask/' + str(train_count) + '.pt')
                            torch.save(torch.tensor([label]), './PredData/train_labels_mask/' + str(train_count) + '.pt')
                            train_count +=1
                        arr = []
                        count +=1
                cap.release()
                cv2.destroyAllWindows()
            label += 1


video_to_array()